In [19]:
%run ../functions_scripts/database_functions.ipynb
%run ../functions_scripts/dataframe_cleaning_functions.ipynb

In [20]:

from pathlib import Path
from typing import List, Optional
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from sqlalchemy import text
from psycopg2.extras import execute_values
from sqlalchemy_utils import database_exists, create_database
import pandas as pd

import os
from dotenv import load_dotenv

load_dotenv()

RAW_DIR = Path(os.getenv("RAW_DIR"))
CLEAN_DIR = Path(os.getenv("CLEAN_DIR"))
ADMIN_DB_URL = os.getenv("ADMIN_DB_URL")
DB_URL = os.getenv("DB_URL")



# 🧹 Nettoyage de la table `caracteristiques`


In [21]:
# Etablir connection avec la base de données
engine = get_engine(DB_URL) 

In [22]:
#Afficher le nombre de lignes dans la table bronze.caracteristiques_raw

query = "SELECT COUNT(*) FROM bronze.caracteristiques_raw"
count= get_single_value(engine, query)
print(f"Nombre de lignes dans bronze.caracteristiques_raw : {count}")
print('\n')

#Récupérer les données dans Dataframe df_caracteristiques
df_caracteristiques = get_df_from_table(engine, "bronze", "caracteristiques_raw")
df_caracteristiques.head()

Nombre de lignes dans bronze.caracteristiques_raw : 475911




,identifiant_de_l_accident,date_et_heure,jour,mois,annee,heure_minute,date,year_georef,lumiere,code_postal,...,localisation,intersection,conditions_atmospheriques,collision,adresse,gps,latitude,longitude,coordonnees,numero
0,201800018092,2018-06-17T17:40:00+02:00,17,06,2018,17:40,None,2018,Plein jour,80150,...,Hors agglomération,1,Normale,Sans collision,D928,Métropole,5019561,0188016,"50.186673, 1.879133",928
1,201600052725,2016-12-07T14:40:00+01:00,07,12,2016,14:40,None,2016,Plein jour,75009,...,En agglomération,2,Normale,Autre collision,"62, RUE DE LA CHAUSSEE D",Métropole,4887530,0233229,"48.875125, 2.332459",62
2,201800018104,2018-02-17T17:00:00+01:00,17,02,2018,17:00,2018-04-01,2018,Plein jour,80370,...,En agglomération,9,Normale,Autre collision,R DE LA VACQUERIE,Métropole,5012380,0216687,"50.128037, 2.161891",None
3,201600054636,2016-08-27T00:01:00+02:00,27,08,2016,00:01,None,2016,Nuit avec éclairage public allumé,75015,...,En agglomération,5,Normale,Deux véhicules – par le coté,"10, RUE DU LAOS",Métropole,4884990,0230110,"48.8501, 2.300974",10
4,201800019309,2018-10-07T19:30:00+02:00,07,10,2018,19:30,2018-09-01,2018,Nuit avec éclairage public allumé,57560,...,En agglomération,1,Normale,Autre collision,119 rue du Général Jordy,Métropole,4863570,0709861,"48.635693, 7.098653",119


In [23]:
# supprimer les colonnes avec plusieurs missing values (dépasse la moitié de valeurs)
df_caracteristiques.drop(columns=['date'], inplace=True)
# creer liste de colonnes supprimées
cols_supprimees= ['date']

In [24]:
# Supprimer les doublons
print(f"Nombre de lignes avant suppression des doublons : {df_caracteristiques.shape[0]}")
df_caracteristiques = df_caracteristiques.drop_duplicates()
print(f"Nombre de lignes après suppression des doublons : {df_caracteristiques.shape[0]}")
#remplacer les valeurs nulles
remplacer_valeurs_nulles(df_caracteristiques)

Nombre de lignes avant suppression des doublons : 475911
Nombre de lignes après suppression des doublons : 475911


,identifiant_de_l_accident,date_et_heure,jour,mois,annee,heure_minute,year_georef,lumiere,code_postal,code_insee,...,localisation,intersection,conditions_atmospheriques,collision,adresse,gps,latitude,longitude,coordonnees,numero
0,201800018092,2018-06-17T17:40:00+02:00,17,06,2018,17:40,2018,Plein jour,80150,80167,...,Hors agglomération,1,Normale,Sans collision,D928,Métropole,5019561,0188016,"50.186673, 1.879133",928
1,201600052725,2016-12-07T14:40:00+01:00,07,12,2016,14:40,2016,Plein jour,75009,75109,...,En agglomération,2,Normale,Autre collision,"62, RUE DE LA CHAUSSEE D",Métropole,4887530,0233229,"48.875125, 2.332459",62
2,201800018104,2018-02-17T17:00:00+01:00,17,02,2018,17:00,2018,Plein jour,80370,80086,...,En agglomération,9,Normale,Autre collision,R DE LA VACQUERIE,Métropole,5012380,0216687,"50.128037, 2.161891",Inconnu
3,201600054636,2016-08-27T00:01:00+02:00,27,08,2016,00:01,2016,Nuit avec éclairage public allumé,75015,75115,...,En agglomération,5,Normale,Deux véhicules – par le coté,"10, RUE DU LAOS",Métropole,4884990,0230110,"48.8501, 2.300974",10
4,201800019309,2018-10-07T19:30:00+02:00,07,10,2018,19:30,2018,Nuit avec éclairage public allumé,57560,57003,...,En agglomération,1,Normale,Autre collision,119 rue du Général Jordy,Métropole,4863570,0709861,"48.635693, 7.098653",119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475906,201600058189,2016-02-18T19:08:00+01:00,18,02,2016,19:08,2016,Nuit avec éclairage public non allumé,97130,97107,...,En agglomération,1,Normale,Autre collision,DELGRES ( BOULEVARD ),Antilles (Martinique ou Guadeloupe),Inconnu,Inconnu,"16.044166, -61.565258",Inconnu
475907,201800017928,2018-05-31T09:30:00+02:00,31,05,2018,09:30,2018,Plein jour,82300,82159,...,Hors agglomération,2,Pluie légère,Trois véhicules et plus – en chaîne,RD964 intersection RD95,Métropole,4412296,0158603,"44.144565, 1.611812",964
475908,201600058951,2016-04-22T07:45:00+02:00,22,04,2016,07:45,2016,Plein jour,97420,97407,...,En agglomération,1,Normale,Deux véhicules – par le coté,COMMUNE DE PARIS (AVENUE,Réunion,Inconnu,Inconnu,"-20.939385, 55.290913",Inconnu
475909,201800018085,2018-07-21T16:55:00+02:00,21,07,2018,16:55,2018,Plein jour,80130,80368,...,Hors agglomération,1,Normale,Autre collision,D925,Métropole,5007348,0154298,"50.086456, 1.542519",925


In [25]:
#Nettoyage sémantique 
#Traiter valeurs spéciales (-1, 00, NULL chaîne).
df_caracteristiques =  traiter_valeurs_speciales(df_caracteristiques)
df_caracteristiques.head()


,identifiant_de_l_accident,date_et_heure,jour,mois,annee,heure_minute,year_georef,lumiere,code_postal,code_insee,...,localisation,intersection,conditions_atmospheriques,collision,adresse,gps,latitude,longitude,coordonnees,numero
0,201800018092,2018-06-17T17:40:00+02:00,17,06,2018,17:40,2018,Plein jour,80150,80167,...,Hors agglomération,1,Normale,Sans collision,D928,Métropole,5019561,0188016,"50.186673, 1.879133",928
1,201600052725,2016-12-07T14:40:00+01:00,07,12,2016,14:40,2016,Plein jour,75009,75109,...,En agglomération,2,Normale,Autre collision,"62, RUE DE LA CHAUSSEE D",Métropole,4887530,0233229,"48.875125, 2.332459",62
2,201800018104,2018-02-17T17:00:00+01:00,17,02,2018,17:00,2018,Plein jour,80370,80086,...,En agglomération,9,Normale,Autre collision,R DE LA VACQUERIE,Métropole,5012380,0216687,"50.128037, 2.161891",None
3,201600054636,2016-08-27T00:01:00+02:00,27,08,2016,00:01,2016,Nuit avec éclairage public allumé,75015,75115,...,En agglomération,5,Normale,Deux véhicules – par le coté,"10, RUE DU LAOS",Métropole,4884990,0230110,"48.8501, 2.300974",10
4,201800019309,2018-10-07T19:30:00+02:00,07,10,2018,19:30,2018,Nuit avec éclairage public allumé,57560,57003,...,En agglomération,1,Normale,Autre collision,119 rue du Général Jordy,Métropole,4863570,0709861,"48.635693, 7.098653",119


In [26]:
#changer type de colonne date_et_heure de texte vers datetime avec fuseau temps UTC+00 mondial
df_caracteristiques['date_et_heure'] =pd.to_datetime(

    df_caracteristiques['date_et_heure'],
    errors='raise',
    utc=True
    ).to_frame()
df_caracteristiques

,identifiant_de_l_accident,date_et_heure,jour,mois,annee,heure_minute,year_georef,lumiere,code_postal,code_insee,...,localisation,intersection,conditions_atmospheriques,collision,adresse,gps,latitude,longitude,coordonnees,numero
0,201800018092,2018-06-17 15:40:00+00:00,17,06,2018,17:40,2018,Plein jour,80150,80167,...,Hors agglomération,1,Normale,Sans collision,D928,Métropole,5019561,0188016,"50.186673, 1.879133",928
1,201600052725,2016-12-07 13:40:00+00:00,07,12,2016,14:40,2016,Plein jour,75009,75109,...,En agglomération,2,Normale,Autre collision,"62, RUE DE LA CHAUSSEE D",Métropole,4887530,0233229,"48.875125, 2.332459",62
2,201800018104,2018-02-17 16:00:00+00:00,17,02,2018,17:00,2018,Plein jour,80370,80086,...,En agglomération,9,Normale,Autre collision,R DE LA VACQUERIE,Métropole,5012380,0216687,"50.128037, 2.161891",None
3,201600054636,2016-08-26 22:01:00+00:00,27,08,2016,00:01,2016,Nuit avec éclairage public allumé,75015,75115,...,En agglomération,5,Normale,Deux véhicules – par le coté,"10, RUE DU LAOS",Métropole,4884990,0230110,"48.8501, 2.300974",10
4,201800019309,2018-10-07 17:30:00+00:00,07,10,2018,19:30,2018,Nuit avec éclairage public allumé,57560,57003,...,En agglomération,1,Normale,Autre collision,119 rue du Général Jordy,Métropole,4863570,0709861,"48.635693, 7.098653",119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475906,201600058189,2016-02-18 18:08:00+00:00,18,02,2016,19:08,2016,Nuit avec éclairage public non allumé,97130,97107,...,En agglomération,1,Normale,Autre collision,DELGRES ( BOULEVARD ),Antilles (Martinique ou Guadeloupe),None,None,"16.044166, -61.565258",None
475907,201800017928,2018-05-31 07:30:00+00:00,31,05,2018,09:30,2018,Plein jour,82300,82159,...,Hors agglomération,2,Pluie légère,Trois véhicules et plus – en chaîne,RD964 intersection RD95,Métropole,4412296,0158603,"44.144565, 1.611812",964
475908,201600058951,2016-04-22 05:45:00+00:00,22,04,2016,07:45,2016,Plein jour,97420,97407,...,En agglomération,1,Normale,Deux véhicules – par le coté,COMMUNE DE PARIS (AVENUE,Réunion,None,None,"-20.939385, 55.290913",None
475909,201800018085,2018-07-21 14:55:00+00:00,21,07,2018,16:55,2018,Plein jour,80130,80368,...,Hors agglomération,1,Normale,Autre collision,D925,Métropole,5007348,0154298,"50.086456, 1.542519",925


In [27]:
#changer le jour , mois en decimale
df_caracteristiques['jour'] = df_caracteristiques['jour'].astype(float) 
df_caracteristiques['mois'] = df_caracteristiques['mois'].astype(float)
#changer l'annee en int
df_caracteristiques['annee'] = df_caracteristiques['annee'].astype(int)

In [28]:
# standardiser la colonne lumiere
#Standardiser casse/accents si nécessaire.
df_caracteristiques = standardiser_colonne_caisse(df_caracteristiques, 'lumiere', lower=True, strip=True, remove_accents=True)

In [29]:
#Harmonisation de la colonne 'intersection'
print(df_caracteristiques.groupby('intersection').size().reset_index(name='counts').sort_values(by='counts', ascending=False))
mapping_intersection = {
    '0': 'non renseigné',
    '1': 'hors intersection',
    '2': 'intersection en x',
    '3': 'intersection en t',
    '4': 'intersection en y',
    '5': 'intersection à plus de 4 branches',
    '6': 'giratoire',
    '7': 'place',
    '8': 'passage à niveau',
    '9': 'autre intersection'
}
harmonise_colonne_df(df_caracteristiques,'intersection',mapping_intersection)
print('\napres harmonisation:')
print(df_caracteristiques.groupby('intersection').size().reset_index(name='counts').sort_values(by='counts', ascending=False))

  intersection  counts
1            1  328937
2            2   56882
3            3   44672
6            6   15386
9            9   10060
4            4    8361
7            7    6100
5            5    4833
8            8     590
0            0      90

apres harmonisation:
                        intersection  counts
2                  hors intersection  328937
4                  intersection en x   56882
3                  intersection en t   44672
1                          giratoire   15386
0                 autre intersection   10060
5                  intersection en y    8361
9                              place    6100
6  intersection à plus de 4 branches    4833
8                   passage à niveau     590
7                      non renseigné      90


In [30]:
# harmoniser colonne collision et remplacer -1 par Inconnu
print(df_caracteristiques.groupby('collision').size().reset_index(name='counts').sort_values(by='counts', ascending=False))

df_caracteristiques = remplacer_valeur(
    df_caracteristiques,
    'collision',
    '-1',
    'Inconnu'
)
print(df_caracteristiques.groupby('collision').size().reset_index(name='counts').sort_values(by='counts', ascending=False))

                                        collision  counts
0                                 Autre collision  161298
2                    Deux véhicules – par le coté  132605
3                  Deux véhicules – par l’arrière   58352
4                                  Sans collision   46028
1                       Deux véhicules - frontale   45192
6             Trois véhicules et plus – en chaîne   16916
5  Trois véhicules et plus - collisions multiples   15508
                                        collision  counts
0                                 Autre collision  161298
2                    Deux véhicules – par le coté  132605
3                  Deux véhicules – par l’arrière   58352
4                                  Sans collision   46028
1                       Deux véhicules - frontale   45192
6             Trois véhicules et plus – en chaîne   16916
5  Trois véhicules et plus - collisions multiples   15508


In [31]:
#convertir la colonne coordonnees en tuples (latitude, longitude) de type float
def parse_coord(x):
    if x is None or pd.isna(x) or ',' not in str(x):
        return (None, None)
    parts = str(x).split(',')
    lat = float(parts[0].strip())
    lon = float(parts[1].strip())
    return (lat, lon)

df_caracteristiques['coordonnees_tuple'] = df_caracteristiques['coordonnees'].apply(parse_coord)
# Vérification
df_caracteristiques[['coordonnees', 'coordonnees_tuple']].head()

,coordonnees,coordonnees_tuple
0,"50.186673, 1.879133","(50.186673, 1.879133)"
1,"48.875125, 2.332459","(48.875125, 2.332459)"
2,"50.128037, 2.161891","(50.128037, 2.161891)"
3,"48.8501, 2.300974","(48.8501, 2.300974)"
4,"48.635693, 7.098653","(48.635693, 7.098653)"


In [32]:

#supprimer les anciens colonne latitude et longitude
df_caracteristiques.drop(columns=['latitude', 'longitude','coordonnees'], inplace=True)

# Créer latitude et longitude à partir de coordonnees_tuple
df_caracteristiques['latitude'] = df_caracteristiques['coordonnees_tuple'].apply(lambda x: x[0] if x else None)
df_caracteristiques['longitude'] = df_caracteristiques['coordonnees_tuple'].apply(lambda x: x[1] if x else None)
#supprimer la colonne coordonnees_tuple
df_caracteristiques.drop(columns=['coordonnees_tuple'], inplace=True)


cols_supprimees.append('coordonnees')
df_caracteristiques.head()

,identifiant_de_l_accident,date_et_heure,jour,mois,annee,heure_minute,year_georef,lumiere,code_postal,code_insee,...,nom_officiel_epci,localisation,intersection,conditions_atmospheriques,collision,adresse,gps,numero,latitude,longitude
0,201800018092,2018-06-17 15:40:00+00:00,17.0,6.0,2018,17:40,2018,plein jour,80150,80167,...,CC Ponthieu-Marquenterre,Hors agglomération,hors intersection,Normale,Sans collision,D928,Métropole,928,50.186673,1.879133
1,201600052725,2016-12-07 13:40:00+00:00,7.0,12.0,2016,14:40,2016,plein jour,75009,75109,...,Métropole du Grand Paris,En agglomération,intersection en x,Normale,Autre collision,"62, RUE DE LA CHAUSSEE D",Métropole,62,48.875125,2.332459
2,201800018104,2018-02-17 16:00:00+00:00,17.0,2.0,2018,17:00,2018,plein jour,80370,80086,...,CC du Territoire Nord Picardie,En agglomération,autre intersection,Normale,Autre collision,R DE LA VACQUERIE,Métropole,None,50.128037,2.161891
3,201600054636,2016-08-26 22:01:00+00:00,27.0,8.0,2016,00:01,2016,nuit avec éclairage public allumé,75015,75115,...,Métropole du Grand Paris,En agglomération,intersection à plus de 4 branches,Normale,Deux véhicules – par le coté,"10, RUE DU LAOS",Métropole,10,48.850100,2.300974
4,201800019309,2018-10-07 17:30:00+00:00,7.0,10.0,2018,19:30,2018,nuit avec éclairage public allumé,57560,57003,...,CC Sarrebourg Moselle Sud,En agglomération,hors intersection,Normale,Autre collision,119 rue du Général Jordy,Métropole,119,48.635693,7.098653


In [33]:
#supprimer les colonnes supprimées de la table caracteristiques_clean dans silver
drop_columns(engine, "silver", "caracteristiques_clean", cols_supprimees)

#inserer les données nettoyées dans la table silver.caracteristiques_clean
truncate_table_if_exists(engine, "silver", "caracteristiques_clean")
# insert_df_to_table(engine, df_caracteristiques, "silver", "caracteristiques_clean")

Colonne supprimée : date
Colonne supprimée : coordonnees
Table caracteristiques_clean.silver vidée si elle existait.


In [34]:
#Fermeture des connexions actives à la base de données
admin_engine = get_engine(ADMIN_DB_URL)
show_open_connections(admin_engine, DB_NAME)
close_all_connections(admin_engine, DB_NAME)
show_open_connections(admin_engine, DB_NAME)


(3774, 'postgres', 'db_accident', 'idle', 'COMMIT')
(3812, 'postgres', 'db_accident', 'idle', 'COMMIT')
(3804, 'postgres', 'db_accident', 'idle', 'select c.oid,pg_catalog.pg_total_relation_size(c.oid) as total_rel_size,pg_catalog.pg_relation_size(c.oid) as rel_size\nFROM pg_class c\nWHERE c.relnamespace=$1')
(3803, 'postgres', 'db_accident', 'idle', 'SHOW search_path')
(3807, 'postgres', 'db_accident', 'idle', 'SELECT * FROM pg_catalog.pg_enum')
(3806, 'postgres', 'db_accident', 'idle', 'SHOW search_path')
(3805, 'postgres', 'db_accident', 'idle', "SELECT c.oid, a.attnum, a.attname, c.relname, n.nspname, a.attnotnull OR (t.typtype = 'd' AND t.typnotnull), a.attidentity != '' OR pg_catalog.pg_get ... (725 characters truncated) ... 5, 16 UNION ALL SELECT 17215, 17 UNION ALL SELECT 17215, 18 UNION ALL SELECT 17215, 19 UNION ALL SELECT 17215, 20 UNION ALL SELECT 17215, 21 UNION AL")


INFO:__main__:Toutes les connexions à 'db_accident' ont été fermées (sauf la session actuelle).
INFO:__main__:Engine SQLAlchemy libéré — plus aucune connexion active.
